## Imperial-UCL Numerics seminar

In [ ]:
include("src/QuadIFS.jl")
using .QuadIFS
using Plots, LaTeXStrings

## Example one: sketching a fractal from its IFS

Generate a set of 10 random points in $(0,1)^2$

In [ ]:
X = rand(2,10)
scatter(X[1,:],X[2,:],legend=:false,markersize=10, 
        markercolor="black",aspect_ratio=:equal)

Define some similarities of the form $s_m(x)=rx+\delta_m$. The syntax describing each map here is ```Similarity(contraction, translation)```.

In [ ]:
s₁ = Similarity(1/3,[0.0,0.0])
s₂ = Similarity(1/3,[2/3,0.0])
s₃ = Similarity(1/3,[0.0,2/3])
s₄ = Similarity(1/3,[2/3,2/3]);

An iterated function system (IFS) can be reprersented as an array of similarities:

In [ ]:
IFS = [s₁, s₂, s₃, s₄];

The full map is defined mathematically as $S(X) := \cup_{m=1}^Ms_m(X)$. We can apply this iteratively to our original set of random points.

In [ ]:
X = full_map(IFS,X)
scatter(X[1,:],X[2,:],legend=:false,markersize=0.5, 
        markercolor="black",aspect_ratio=:equal)

## Example two: quadrature rule on an IFS attractor

In [ ]:
Γ = Attractor(IFS);

In [ ]:
h = 0.01
x,w = barycentre_rule(Γ,h);

In [ ]:
scatter(x[:,1],x[:,2],legend=:false,markersize=1, 
        markercolor="black",aspect_ratio=:equal)

## Example three: Acoustic scattering

In [ ]:
k = 53 # wavenumber
d = [0 0 -1] # incoming wave direction
f(x) = exp(im*k*(d[1:2]'*x)); # RHS data

In [ ]:
S = SingleLayer(Γ, k); # single layer integral operator defined on fractal Γ

In [ ]:
h_BEM = 0.01
h_QUAD = 0.001
Sₕ = DiscreteBIO(S,h_BEM,h_QUAD); #discretise BIO

The below is a temporary fix for Julia 1.7, please ignore... details are [here](https://github.com/JuliaLang/julia/issues/43301) and [here](https://github.com/JuliaLang/julia/issues/43008)

In [ ]:
using LinearAlgebra
BLAS.set_num_threads(1)

In [ ]:
ϕₕ = Sₕ\f; # solve the discrete problem

The rest of the code is just making a slice plot...

In [ ]:
h=0.01 # adujust this for more acurate / faster plots
z_val = - 0.1
x_1 = [j for j=0:h:1] # !!! ADJUST THESE IF fractal is outside [0,1]²
x_2 = [j for j=0:h:1]
X_1 = repeat(x_1, outer=(1,length(x_2)))
X_2 = copy(transpose(repeat(x_2, outer=(1,length(x_1)))))
N = length(x_1)*length(x_2)
X = [reshape(X_1, (N,)) reshape(X_2, (N,)) z_val*ones(Float64,N)]
uˢ = - single_layer_potential(Γ,k,ϕₕ,X)
uⁱ(x) = exp.(im*k*(x*d'))[:] # define inc field in domain
uᵗ = uⁱ(X) + uˢ
vals = reshape(uᵗ,(length(x_1),length(x_2)));

In [ ]:
heatmap(x_1,x_2,transpose(abs.(uˢ)),  aspect_ratio = 1, 
    title=L"|u^s|", legend=true, c = :thermal)